# ENVIRONMENT SETUP

In [ ]:
!pip install -q transformers==4.45.0
!pip install -q datasets==2.16.1
!pip install -q accelerate==1.0.0
!pip install -q evaluate==0.4.3
!pip install -q jiwer==3.0.3
!pip install -q tensorboard
!pip install -q soundfile
!pip install -q librosa

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 83.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 77.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.4/166.4 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 10.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
pathos 0.3.2 requires dill>=0.3.8, but you have dill 0.3.7 which is incompatible.
pathos 0.3.2 requires multiprocess>=0.70.16, but you have multiprocess 0.70.15 which is in

# SYSTEM VERIFICATION

In [ ]:

import os
import torch
import transformers
import accelerate
import gc

print("=" * 70)
print("SYSTEM VERIFICATION")
print("=" * 70)
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
torch.cuda.set_device(0)

print("\n🔧 GPU Configuration:")
print(f"   CUDA_VISIBLE_DEVICES: {os.environ.get('CUDA_VISIBLE_DEVICES')}")
print(f"   Current Device: {torch.cuda.current_device()}")
print(f"\n📦 Library Versions:")
print(f"   PyTorch: {torch.__version__}")
print(f"   Transformers: {transformers.__version__}")
print(f"   Accelerate: {accelerate.__version__}")

print(f"\n🖥️ Hardware Status:")
print(f"   CUDA Available: {torch.cuda.is_available()}")
print(f"   Number of GPUs Visible: {torch.cuda.device_count()}")

if torch.cuda.is_available():
    print(f"   GPU Name: {torch.cuda.get_device_name(0)}")
    print(f"   GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")

gc.collect()
if torch.cuda.is_available():
    torch.cuda.empty_cache()

print("\n✅ All systems verified and memory cleared")

print("\n" + "=" * 70)
print("DETAILED GPU INFORMATION")
print("=" * 70 + "\n")

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)

if gpu_info.find('failed') >= 0:
    print('❌ Not connected to a GPU')
else:
    print(gpu_info)

SYSTEM VERIFICATION

🔧 GPU Configuration:
   CUDA_VISIBLE_DEVICES: 0
   Current Device: 0

📦 Library Versions:
   PyTorch: 2.6.0+cu124
   Transformers: 4.45.0
   Accelerate: 1.0.0

🖥️ Hardware Status:
   CUDA Available: True
   Number of GPUs Visible: 1
   GPU Name: Tesla T4
   GPU Memory: 14.74 GB

✅ All systems verified and memory cleared

DETAILED GPU INFORMATION

Mon Nov 17 07:54:51 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+===

# DATA PREPARATION

## Download and Extract Dataset

In [ ]:
print("\n" + "=" * 70)
print("DATA PREPARATION")
print("=" * 70)

# Install huggingface_hub for dataset download
!pip install -q huggingface_hub

from huggingface_hub import list_repo_files, hf_hub_download
import shutil

print("\n📥 Downloading dataset from Hugging Face...")

# Repository configuration
repo_id = "bitwisemind/hackathon"
repo_type = "dataset"
download_dir = "./"

# List and download all files
files = list_repo_files(repo_id=repo_id, repo_type=repo_type)
print(f"📁 Found {len(files)} files in the repository")

os.makedirs(download_dir, exist_ok=True)
downloaded_files = []

for file in files:
    file_path = hf_hub_download(repo_id=repo_id, filename=file, repo_type=repo_type)
    local_path = os.path.join(download_dir, file)
    os.makedirs(os.path.dirname(local_path), exist_ok=True)
    shutil.copy(file_path, local_path)
    downloaded_files.append(local_path)

print(f"✅ All files downloaded to: {download_dir}")



DATA PREPARATION

📥 Downloading dataset from Hugging Face...
📁 Found 5 files in the repository


.gitattributes: 0.00B [00:00, ?B/s]

README.md:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

Test.zip:   0%|          | 0.00/63.0M [00:00<?, ?B/s]

Train.zip:   0%|          | 0.00/289M [00:00<?, ?B/s]

Train_annotation.zip:   0%|          | 0.00/58.5k [00:00<?, ?B/s]

✅ All files downloaded to: ./


## Extract Downloaded Archives

In [ ]:
print("\n📦 Extracting archives...")

!unzip -q ./Train.zip
!unzip -q ./Train_annotation.zip
!unzip -q ./Test.zip

print("✅ Archives extracted successfully")


📦 Extracting archives...
✅ Archives extracted successfully


## Clean Up Zip Files (Only for Kaggle)

In [ ]:
print("\n🗑️ Cleaning up zip files...")

os.remove("/kaggle/working/Test.zip")
os.remove("/kaggle/working/Train.zip")
os.remove("/kaggle/working/Train_annotation.zip")

print("✅ Cleanup complete")


🗑️ Cleaning up zip files...
✅ Cleanup complete


## Define Dataset Configuration

In [ ]:
# Dataset paths
AUDIO_BASE_PATH = "./Train/"
ANNOTATION_BASE_PATH = "./Train_annotation/"

# List of all 20 Bengali regional dialects
REGIONS = [
    "Barisal", "Bhola", "Bogura", "Brahmanbaria", "Chittagong", "Comilla",
    "Dhaka", "Feni", "Jessore", "Jhenaidah", "Khulna", "Kushtia",
    "Lakshmipur", "Mymensingh", "Natore", "Noakhali", "Pabna", "Rajshahi",
    "Rangpur", "Sylhet"
]

print(f"\n📍 Configured for {len(REGIONS)} regional dialects")
print(f"   Audio path: {AUDIO_BASE_PATH}")
print(f"   Annotation path: {ANNOTATION_BASE_PATH}")


📍 Configured for 20 regional dialects
   Audio path: ./Train/
   Annotation path: ./Train_annotation/


# IMPORT REQUIRED LIBRARIES

In [ ]:
import pandas as pd
import numpy as np
from datasets import Dataset, DatasetDict, Audio
import warnings

warnings.filterwarnings('ignore')

print("\n✅ Data processing libraries imported")

from transformers import (
    WhisperFeatureExtractor,
    WhisperTokenizer,
    WhisperProcessor,
    WhisperForConditionalGeneration,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer
)

print("✅ Transformers components imported")

from dataclasses import dataclass
from typing import Any, Dict, List, Union
import evaluate

print("✅ Training utilities imported")


✅ Data processing libraries imported


2025-11-17 07:55:15.043495: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763366115.216776      20 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763366115.267601      20 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

✅ Transformers components imported
✅ Training utilities imported


# DATASET LOADING

## Define Dataset Loading Function

In [ ]:
def load_custom_dataset(audio_base_path, annotation_base_path, regions, split_ratio=0.9):
    """
    Load custom Bengali regional ASR dataset from audio files and CSV annotations

    Args:
        audio_base_path: Base path to audio files
        annotation_base_path: Base path to annotation CSV files
        regions: List of region names
        split_ratio: Train/test split ratio (default: 0.9)

    Returns:
        DatasetDict with 'train' and 'test' splits
    """
    all_data = []

    for region in regions:
        csv_path = os.path.join(annotation_base_path, f"{region}.csv")
        audio_folder = os.path.join(audio_base_path, region)

        # Check if files exist
        if not os.path.exists(csv_path):
            print(f"⚠️ CSV file not found for {region}")
            continue

        if not os.path.exists(audio_folder):
            print(f"⚠️ Audio folder not found for {region}")
            continue

        try:
            df = pd.read_csv(csv_path)
        except Exception as e:
            print(f"❌ Error reading CSV for {region}: {e}")
            continue

        # Process each row in the CSV
        for idx, row in df.iterrows():
            try:
                audio_filename = str(row.iloc[0]).strip()
                transcription = str(row.iloc[1]).strip()

                # Skip invalid transcriptions
                if not transcription or transcription == 'nan' or pd.isna(transcription):
                    continue

                audio_path = os.path.join(audio_folder, audio_filename)

                # Check if audio file exists and is not empty
                if os.path.exists(audio_path) and os.path.getsize(audio_path) > 0:
                    all_data.append({
                        "path": audio_path,
                        "sentence": transcription
                    })
            except Exception as e:
                continue

        # Clean up DataFrame to free memory
        del df
        gc.collect()

    print(f"📊 Total samples loaded: {len(all_data)}")

    if len(all_data) == 0:
        raise ValueError("❌ No valid samples found. Please check your dataset paths and files.")

    # Split into train and test
    split_idx = int(len(all_data) * split_ratio)
    if split_idx == 0:
        split_idx = 1
    if split_idx >= len(all_data):
        split_idx = len(all_data) - 1

    train_data = all_data[:split_idx]
    test_data = all_data[split_idx:]

    # Create datasets
    train_dict = {
        "path": [item["path"] for item in train_data],
        "sentence": [item["sentence"] for item in train_data]
    }

    test_dict = {
        "path": [item["path"] for item in test_data],
        "sentence": [item["sentence"] for item in test_data]
    }

    train_dataset = Dataset.from_dict(train_dict)
    test_dataset = Dataset.from_dict(test_dict)

    # Clear temporary data
    del all_data, train_data, test_data, train_dict, test_dict
    gc.collect()

    # Cast to audio format
    train_dataset = train_dataset.cast_column("path", Audio(sampling_rate=16000))
    test_dataset = test_dataset.cast_column("path", Audio(sampling_rate=16000))

    # Rename column
    train_dataset = train_dataset.rename_column("path", "audio")
    test_dataset = test_dataset.rename_column("path", "audio")

    return DatasetDict({
        "train": train_dataset,
        "test": test_dataset
    })

print("✅ Dataset loading function defined")

✅ Dataset loading function defined


## Load the Dataset

In [ ]:
print("\n" + "=" * 70)
print("LOADING DATASET")
print("=" * 70 + "\n")

custom_voice = load_custom_dataset(AUDIO_BASE_PATH, ANNOTATION_BASE_PATH, REGIONS)

print("\n📊 Dataset Structure:")
print(custom_voice)

# Memory cleanup
gc.collect()
if torch.cuda.is_available():
    torch.cuda.empty_cache()


LOADING DATASET

📊 Total samples loaded: 3350

📊 Dataset Structure:
DatasetDict({
    train: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 3015
    })
    test: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 335
    })
})


# MODEL AND PROCESSOR SETUP
## We utilized [`bengaliAI/tugstugi_bengaliai-regional-asr_whisper-medium`](https://huggingface.co/bengaliAI/tugstugi_bengaliai-regional-asr_whisper-medium) model which originally [`openai/whisper-medium`](https://huggingface.co/openai/whisper-medium) model, finetuned in [`Ben10`](https://www.kaggle.com/competitions/ben10) Dataset.

In [ ]:
print("\n" + "=" * 70)
print("LOADING WHISPER PROCESSOR")
print("=" * 70 + "\n")

model_name = "bengaliAI/tugstugi_bengaliai-regional-asr_whisper-medium"

print(f"📦 Loading from: {model_name}")

feature_extractor = WhisperFeatureExtractor.from_pretrained(model_name)
tokenizer = WhisperTokenizer.from_pretrained(
    model_name,
    language="Bengali",
    task="transcribe"
)
processor = WhisperProcessor.from_pretrained(
    model_name,
    language="Bengali",
    task="transcribe"
)

print("✅ Processor loaded successfully")


LOADING WHISPER PROCESSOR

📦 Loading from: bengaliAI/tugstugi_bengaliai-regional-asr_whisper-medium


preprocessor_config.json:   0%|          | 0.00/339 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

normalizer.json: 0.00B [00:00, ?B/s]

added_tokens.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

✅ Processor loaded successfully


# DATA PREPROCESSING

## Define Preprocessing Function

In [ ]:
def prepare_dataset(batch):
    """
    Prepare dataset batch for training

    - Converts audio to log-Mel spectrogram features
    - Tokenizes transcriptions to label IDs

    Args:
        batch: Single batch from dataset

    Returns:
        Processed batch with input_features and labels
    """
    audio = batch["audio"]

    # Compute log-Mel input features from input audio array
    batch["input_features"] = feature_extractor(
        audio["array"],
        sampling_rate=audio["sampling_rate"]
    ).input_features[0]

    # Encode target text to label ids
    batch["labels"] = tokenizer(batch["sentence"]).input_ids

    return batch

print("✅ Preprocessing function defined")

✅ Preprocessing function defined


## Preprocess the Dataset

In [ ]:
print("\n" + "=" * 70)
print("PREPROCESSING DATASET")
print("=" * 70 + "\n")

print("⏳ Processing dataset (this may take a while)...")

custom_voice = custom_voice.map(
    prepare_dataset,
    remove_columns=custom_voice.column_names["train"],
    desc="Preparing dataset",
    batched=False,
    num_proc=1
)

print("\n✅ Dataset preprocessing complete")
print(f"\n📊 Processed Dataset Structure:")
print(custom_voice)

# Memory cleanup
gc.collect()
if torch.cuda.is_available():
    torch.cuda.empty_cache()


PREPROCESSING DATASET

⏳ Processing dataset (this may take a while)...


Preparing dataset:   0%|          | 0/3015 [00:00<?, ? examples/s]

Preparing dataset:   0%|          | 0/335 [00:00<?, ? examples/s]


✅ Dataset preprocessing complete

📊 Processed Dataset Structure:
DatasetDict({
    train: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 3015
    })
    test: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 335
    })
})


# DATA COLLATOR

In [ ]:
@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    """
    Data collator for speech-to-text models

    - Pads input features to same length
    - Pads labels and masks padding tokens with -100
    - Removes BOS token from labels if present
    """
    processor: Any
    decoder_start_token_id: int

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # Split inputs and labels
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # Get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # Replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # If bos token is appended in previous tokenization step, cut bos token here
        if (labels[:, 0] == self.decoder_start_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

print("✅ Data collator class defined")

✅ Data collator class defined


# MODEL LOADING

## Pre-Load Memory Cleanup

In [ ]:
print("\n" + "=" * 70)
print("LOADING WHISPER MODEL")
print("=" * 70 + "\n")

print("🧹 Clearing GPU memory before model load...")

gc.collect()
if torch.cuda.is_available():
    torch.cuda.empty_cache()


LOADING WHISPER MODEL

🧹 Clearing GPU memory before model load...


## Load Pre-trained Whisper Model

In [ ]:
print(f"📦 Loading model: {model_name}")

model = WhisperForConditionalGeneration.from_pretrained(
    model_name,
    use_cache=True,
    low_cpu_mem_usage=True
)

📦 Loading model: bengaliAI/tugstugi_bengaliai-regional-asr_whisper-medium


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/3.06G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/223 [00:00<?, ?B/s]

## Fix Language/Task Configuration

In [ ]:
print("\n🔧 Fixing generation config...")

# Remove language/task attributes that cause errors
if hasattr(model.generation_config, 'language'):
    delattr(model.generation_config, 'language')
if hasattr(model.generation_config, 'task'):
    delattr(model.generation_config, 'task')

# Ensure forced_decoder_ids stays None
model.generation_config.forced_decoder_ids = None


🔧 Fixing generation config...


## Verify Model Configuration

In [ ]:
print("\n✅ Model loaded successfully")

if torch.cuda.is_available():
    print(f"💾 GPU Memory after model load: {torch.cuda.memory_allocated(0) / 1024**3:.2f} GB")

print(f"\n🔍 Generation Config Verification:")
print(f"   forced_decoder_ids: {model.generation_config.forced_decoder_ids}")
print(f"   Has language attr: {hasattr(model.generation_config, 'language')}")
print(f"   Has task attr: {hasattr(model.generation_config, 'task')}")
print(f"   Has lang_to_id: {hasattr(model.generation_config, 'lang_to_id')}")
print("✅ Config is clean - no language forcing will occur during generation")



✅ Model loaded successfully
💾 GPU Memory after model load: 0.00 GB

🔍 Generation Config Verification:
   forced_decoder_ids: None
   Has language attr: False
   Has task attr: False
   Has lang_to_id: False
✅ Config is clean - no language forcing will occur during generation


## Initialize Data Collator Instance

In [ ]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(
    processor=processor,
    decoder_start_token_id=model.config.decoder_start_token_id,
)

print("\n✅ Data collator initialized")


✅ Data collator initialized


# EVALUATION METRICS

## Load WER Metric

In [ ]:
print("\n" + "=" * 70)
print("LOADING EVALUATION METRICS")
print("=" * 70 + "\n")

print("📊 Loading WER (Word Error Rate) metric...")

try:
    metric = evaluate.load("wer", trust_remote_code=True)
    print("✅ WER metric loaded from evaluate")
except Exception as e:
    print(f"⚠️ Could not load from evaluate: {e}")
    print("🔄 Using jiwer as fallback...")

    import jiwer

    class WERMetric:
        """Wrapper for jiwer to match evaluate interface"""
        def compute(self, predictions, references):
            if isinstance(predictions, str):
                predictions = [predictions]
            if isinstance(references, str):
                references = [references]
            return jiwer.wer(references, predictions)

    metric = WERMetric()
    print("✅ Using jiwer for WER calculation")


LOADING EVALUATION METRICS

📊 Loading WER (Word Error Rate) metric...


✅ WER metric loaded from evaluate


## Define Compute Metrics Function

In [ ]:
def compute_metrics(pred):
    """
    Compute Word Error Rate (WER) for model predictions

    Args:
        pred: Predictions object with predictions and labels

    Returns:
        Dictionary with WER score (lower is better)
    """
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # Replace -100 with pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # Decode predictions and labels
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    # Compute WER
    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

print("✅ Compute metrics function defined")

✅ Compute metrics function defined


# TRAINING CONFIGURATION

## Define Training Arguments

In [ ]:
print("\n" + "=" * 70)
print("CONFIGURING TRAINING")
print("=" * 70 + "\n")

training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-medium-bengali-regional",

    # ========== Batch Configuration ==========
    per_device_train_batch_size=4,        # Batch size per GPU
    gradient_accumulation_steps=4,         # Accumulate gradients
    per_device_eval_batch_size=4,         # Eval batch size

    # ========== Learning Rate ==========
    learning_rate=1e-5,                    # Conservative learning rate
    warmup_steps=200,                      # Warmup for stable training

    # ========== Training Duration ==========
    max_steps=4000,                        # Total training steps (~3.5-4 hours)

    # ========== Memory Optimization ==========
    gradient_checkpointing=True,           # Save memory with checkpointing
    fp16=True,                             # Use mixed precision
    fp16_opt_level="O1",                   # FP16 optimization level

    # ========== Evaluation Settings ==========
    eval_strategy="steps",                 # Evaluate at fixed steps
    predict_with_generate=True,            # Generate predictions for WER
    generation_max_length=225,             # Max generation length

    # ========== Checkpoint Management ==========
    save_steps=1000,                       # Save every 1000 steps
    eval_steps=1000,                       # Evaluate every 1000 steps
    logging_steps=50,                      # Log every 50 steps
    save_total_limit=1,                    # Keep only latest checkpoint
    load_best_model_at_end=True,           # Load best model after training
    metric_for_best_model="wer",           # Use WER for best model
    greater_is_better=False,               # Lower WER is better

    # ========== Data Loading ==========
    dataloader_num_workers=2,              # Parallel data loading
    dataloader_pin_memory=True,            # Pin memory for faster transfer

    # ========== Optimizer ==========
    optim="adamw_torch",                   # AdamW optimizer
    max_grad_norm=1.0,                     # Gradient clipping

    # ========== Logging and Output ==========
    report_to=["tensorboard"],             # Use TensorBoard for monitoring
    push_to_hub=False,                     # Don't push to Hugging Face Hub

    # ========== GPU Configuration ==========
    no_cuda=False,                         # Use CUDA
    local_rank=-1,                         # Disable distributed training
)

print("✅ Training arguments configured")

print("\n📋 Training Configuration Summary:")
print(f"   Output Directory: {training_args.output_dir}")
print(f"   Batch Size: {training_args.per_device_train_batch_size}")
print(f"   Gradient Accumulation: {training_args.gradient_accumulation_steps}")
print(f"   Effective Batch Size: {training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps}")
print(f"   Learning Rate: {training_args.learning_rate}")
print(f"   Max Steps: {training_args.max_steps}")
print(f"   FP16: {training_args.fp16}")
print(f"   Gradient Checkpointing: {training_args.gradient_checkpointing}")
print(f"   Save Total Limit: {training_args.save_total_limit} (only latest checkpoint)")



CONFIGURING TRAINING

✅ Training arguments configured

📋 Training Configuration Summary:
   Output Directory: ./whisper-medium-bengali-regional
   Batch Size: 4
   Gradient Accumulation: 4
   Effective Batch Size: 16
   Learning Rate: 1e-05
   Max Steps: 4000
   FP16: True
   Gradient Checkpointing: True
   Save Total Limit: 1 (only latest checkpoint)


## Verify Single GPU Configuration

In [ ]:
print("\n🔍 Verifying GPU Configuration:")
print(f"   CUDA_VISIBLE_DEVICES: {os.environ.get('CUDA_VISIBLE_DEVICES', 'Not set')}")
print(f"   Number of GPUs PyTorch sees: {torch.cuda.device_count()}")
print(f"   Current device: {torch.cuda.current_device()}")

gc.collect()
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    print(f"   GPU Memory allocated: {torch.cuda.memory_allocated(0) / 1024**3:.2f} GB")
    print(f"   GPU Memory reserved: {torch.cuda.memory_reserved(0) / 1024**3:.2f} GB")



🔍 Verifying GPU Configuration:
   CUDA_VISIBLE_DEVICES: 0
   Number of GPUs PyTorch sees: 1
   Current device: 0
   GPU Memory allocated: 0.00 GB
   GPU Memory reserved: 0.00 GB


# CUSTOM CALLBACKS

In [ ]:
from transformers import TrainerCallback

class MemoryCleanupCallback(TrainerCallback):
    """
    Custom callback for memory management and checkpoint cleanup

    Features:
    - Periodic memory cleanup during training
    - Automatic removal of old checkpoints (keeps only latest)
    - Memory cleanup before evaluation
    """

    def on_step_end(self, args, state, control, **kwargs):
        """Clean memory after each training step"""
        if state.global_step % 10 == 0:  # Every 10 steps
            gc.collect()
            if torch.cuda.is_available():
                torch.cuda.empty_cache()
        return control

    def on_save(self, args, state, control, **kwargs):
        """Clean up old checkpoints after saving"""
        # Remove old checkpoints except the latest
        if os.path.exists(args.output_dir):
            checkpoints = [d for d in os.listdir(args.output_dir) if d.startswith("checkpoint-")]
            if len(checkpoints) > 1:
                # Sort by checkpoint number
                checkpoints_sorted = sorted(checkpoints, key=lambda x: int(x.split("-")[1]))
                # Keep only the last checkpoint
                for checkpoint in checkpoints_sorted[:-1]:
                    checkpoint_path = os.path.join(args.output_dir, checkpoint)
                    try:
                        shutil.rmtree(checkpoint_path)
                        print(f"🗑️ Removed old checkpoint: {checkpoint}")
                    except Exception as e:
                        print(f"⚠️ Could not remove {checkpoint}: {e}")

        # Clean memory after saving
        gc.collect()
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
        return control

    def on_evaluate(self, args, state, control, **kwargs):
        """Clean memory before evaluation"""
        gc.collect()
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
        return control

memory_callback = MemoryCleanupCallback()
print("✅ Memory cleanup callback created with checkpoint management")


✅ Memory cleanup callback created with checkpoint management


# TRAINER INITIALIZATION

## Initialize Seq2SeqTrainer

In [ ]:
print("\n" + "=" * 70)
print("INITIALIZING TRAINER")
print("=" * 70 + "\n")

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=custom_voice["train"],
    eval_dataset=custom_voice["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
    callbacks=[memory_callback],
)

print("✅ Trainer initialized successfully")
print("   - Model: Whisper Medium (Bengali Regional)")
print("   - Training samples:", len(custom_voice["train"]))
print("   - Evaluation samples:", len(custom_voice["test"]))
print("   - Memory optimization: Enabled")
print("   - Checkpoint management: Enabled")


INITIALIZING TRAINER



max_steps is given, it will override any value given in num_train_epochs


✅ Trainer initialized successfully
   - Model: Whisper Medium (Bengali Regional)
   - Training samples: 3015
   - Evaluation samples: 335
   - Memory optimization: Enabled
   - Checkpoint management: Enabled


## Save Processor Configuration

In [ ]:
print("\n💾 Saving processor configuration...")
processor.save_pretrained(training_args.output_dir)
print(f"✅ Processor saved to: {training_args.output_dir}")


💾 Saving processor configuration...
✅ Processor saved to: ./whisper-medium-bengali-regional


# TRAINING EXECUTION

## Display Training Summary

In [ ]:
print("\n" + "=" * 70)
print("TRAINING SUMMARY")
print("=" * 70)

print(f"\n📊 Dataset Statistics:")
print(f"   Training samples: {len(custom_voice['train'])}")
print(f"   Evaluation samples: {len(custom_voice['test'])}")

print(f"\n⚙️ Training Configuration:")
print(f"   Per-device batch size: {training_args.per_device_train_batch_size}")
print(f"   Gradient accumulation steps: {training_args.gradient_accumulation_steps}")
print(f"   Effective batch size: {training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps}")
print(f"   Learning rate: {training_args.learning_rate}")
print(f"   Total training steps: {training_args.max_steps}")
print(f"   Evaluation frequency: Every {training_args.eval_steps} steps")
print(f"   Checkpoint limit: {training_args.save_total_limit} (latest only)")

print(f"\n⏱️ Estimated Training Time:")
print(f"   Approximate duration: 3.5-4 hours")
print(f"   Evaluations: {training_args.max_steps // training_args.eval_steps} times")

print(f"\n💾 Memory Management:")
print(f"   FP16 training: Enabled")
print(f"   Gradient checkpointing: Enabled")
print(f"   Automatic checkpoint cleanup: Enabled")


TRAINING SUMMARY

📊 Dataset Statistics:
   Training samples: 3015
   Evaluation samples: 335

⚙️ Training Configuration:
   Per-device batch size: 4
   Gradient accumulation steps: 4
   Effective batch size: 16
   Learning rate: 1e-05
   Total training steps: 4000
   Evaluation frequency: Every 1000 steps
   Checkpoint limit: 1 (latest only)

⏱️ Estimated Training Time:
   Approximate duration: 3.5-4 hours
   Evaluations: 4 times

💾 Memory Management:
   FP16 training: Enabled
   Gradient checkpointing: Enabled
   Automatic checkpoint cleanup: Enabled


## Configure Memory Allocation

In [ ]:
print("\n🔧 Configuring memory allocation...")
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

if torch.cuda.is_available():
    torch.cuda.empty_cache()
    gc.collect()
    print(f"   Initial GPU Memory: {torch.cuda.memory_allocated(0) / 1024**3:.2f} GB")
    print(f"   GPU Memory Reserved: {torch.cuda.memory_reserved(0) / 1024**3:.2f} GB")



🔧 Configuring memory allocation...
   Initial GPU Memory: 2.85 GB
   GPU Memory Reserved: 2.86 GB


## START TRAINING

In [ ]:
print("\n" + "=" * 70)
print("🚀 STARTING TRAINING")
print("=" * 70 + "\n")

print("⏳ Training in progress... This will take approximately 3.5-4 hours")
print("📊 You can monitor progress in TensorBoard")
print("💡 Tip: Keep this tab open to track training progress\n")

try:
    # Start training
    trainer.train()

    print("\n" + "=" * 70)
    print("✅ TRAINING COMPLETED SUCCESSFULLY!")
    print("=" * 70)

except Exception as e:
    error_msg = str(e)
    print(f"\n{'=' * 70}")
    print("❌ TRAINING ERROR")
    print("=" * 70)
    print(f"\nError: {error_msg}\n")

    # Provide specific solutions based on error type
    if "out of memory" in error_msg.lower():
        print("🔧 SOLUTION FOR OUT OF MEMORY ERROR:")
        print("   Reduce batch size in Cell 11.1:")
        print("   Change: per_device_train_batch_size=2")
        print("   Change: gradient_accumulation_steps=8")
        print("   This maintains effective batch size while using less memory\n")

    elif "unscale" in error_msg.lower() or "fp16" in error_msg.lower():
        print("🔧 SOLUTION FOR FP16 ERROR:")
        print("   This error shouldn't occur with the current configuration")
        print("   If it persists, try setting fp16=False in Cell 11.1\n")

    # Print full traceback
    import traceback
    print("Full error traceback:")
    traceback.print_exc()

    # Clean up memory
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

    raise  # Re-raise the exception to stop execution


🚀 STARTING TRAINING

⏳ Training in progress... This will take approximately 3.5-4 hours
📊 You can monitor progress in TensorBoard
💡 Tip: Keep this tab open to track training progress



Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.43.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...


Step,Training Loss,Validation Loss,Wer
1000,0.008100,0.064153,10.964187
2000,0.001300,0.055127,7.107438
3000,0.000900,0.057683,7.878788
4000,0.000400,0.050029,6.170799


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


🗑️ Removed old checkpoint: checkpoint-1000
🗑️ Removed old checkpoint: checkpoint-2000
🗑️ Removed old checkpoint: checkpoint-3000


There were missing keys in the checkpoint model loaded: ['proj_out.weight'].



✅ TRAINING COMPLETED SUCCESSFULLY!


# MODEL SAVING

## Pre-Save Memory Cleanup

In [ ]:
print("\n" + "=" * 70)
print("SAVING FINAL MODEL")
print("=" * 70 + "\n")

print("🧹 Cleaning up memory before saving...")
gc.collect()
if torch.cuda.is_available():
    torch.cuda.empty_cache()


SAVING FINAL MODEL

🧹 Cleaning up memory before saving...


## Save Final Model and Processor

In [ ]:
print("💾 Saving final model and processor...")

final_model_path = "./whisper-medium-bengali-regional-final"

trainer.save_model(final_model_path)
processor.save_pretrained(final_model_path)

print(f"✅ Model saved successfully to: {final_model_path}")

💾 Saving final model and processor...
✅ Model saved successfully to: ./whisper-medium-bengali-regional-final


## Clean Up Training Checkpoints

In [ ]:
print("\n🗑️ Cleaning up intermediate checkpoints...")

checkpoint_count = 0
if os.path.exists(training_args.output_dir):
    checkpoints = [d for d in os.listdir(training_args.output_dir)
                   if d.startswith("checkpoint-")]

    for checkpoint in checkpoints:
        checkpoint_path = os.path.join(training_args.output_dir, checkpoint)
        try:
            shutil.rmtree(checkpoint_path)
            checkpoint_count += 1
            print(f"   ✓ Removed: {checkpoint}")
        except Exception as e:
            print(f"   ✗ Could not remove {checkpoint}: {e}")

if checkpoint_count > 0:
    print(f"\n✅ Removed {checkpoint_count} intermediate checkpoint(s)")
else:
    print("\n✓ No intermediate checkpoints to remove")



🗑️ Cleaning up intermediate checkpoints...
   ✓ Removed: checkpoint-4000

✅ Removed 1 intermediate checkpoint(s)


# TRAINING RESULTS SUMMARY

In [ ]:
print("\n" + "=" * 70)
print("TRAINING RESULTS SUMMARY")
print("=" * 70 + "\n")

print("✅ Training completed successfully!\n")

print("📊 Final Model Information:")
print(f"   Model path: {final_model_path}")
print(f"   Architecture: Whisper Medium")
print(f"   Language: Bengali (20 regional dialects)")
print(f"   Training samples: {len(custom_voice['train'])}")
print(f"   Evaluation samples: {len(custom_voice['test'])}")
print(f"   Training steps completed: {training_args.max_steps}")

print("\n💾 Saved Components:")
print(f"   ✓ Model weights")
print(f"   ✓ Model configuration")
print(f"   ✓ Processor (tokenizer + feature extractor)")
print(f"   ✓ Generation config")

print("\n📈 Evaluation Metrics:")
print("   Check the final WER (Word Error Rate) score above")
print("   Lower WER = better performance")


TRAINING RESULTS SUMMARY

✅ Training completed successfully!

📊 Final Model Information:
   Model path: ./whisper-medium-bengali-regional-final
   Architecture: Whisper Medium
   Language: Bengali (20 regional dialects)
   Training samples: 3015
   Evaluation samples: 335
   Training steps completed: 4000

💾 Saved Components:
   ✓ Model weights
   ✓ Model configuration
   ✓ Processor (tokenizer + feature extractor)
   ✓ Generation config

📈 Evaluation Metrics:
   Check the final WER (Word Error Rate) score above
   Lower WER = better performance


# END OF TRAINING